# Display Main Agent Graph

In [ ]:
"""
Middleware Overview:
- PatchToolCallsMiddleware: This is default in-built middleware. Normalizes tool calls into the shape Deep Agents expects for 
  routing and message formatting. Built-in default to handle cases where the deep agent 
  is downstream of other agents.
  
- SummarizationMiddleware: This is default in-built middleware. Compacts conversation context before sending to the model when 
  token thresholds are exceeded, maintaining efficiency in long conversations.
  
- MemoryCleanupMiddleware: Prunes long-term memory to stay within token limits. This is 
  custom middleware we defined manually for this agent.

Note: Using xray=1 to show sub-agent internal graphs. Without xray, sub-agents appear 
as tools and their internal structure is hidden.
"""

import sys
sys.path.insert(0, '..')  # Add parent directory to path

from IPython.display import Image, display
from agents.main_agent import agent

# xray=1 shows the internal structure of sub-agents
display(Image(agent.get_graph(xray=1).draw_mermaid_png()))

# Deep Research Agent - Memory Management

This notebook allows you to:
1. View what's in the agent's long-term memory
2. Add memories manually (e.g., pre-seed website quality ratings)
3. Update or delete memories
4. Search memories semantically

## Prerequisites

This notebook connects to the LangGraph server, which provides:
- **Cloud-hosted store** - Persistent memory across sessions (same as LangSmith Studio)
- **Cloud-hosted checkpointer** - Conversation state persistence

### Setup Steps

1. **Start the LangGraph server** (from the `deep-agent/` directory):
   ```bash
   cd deep-agent
   langgraph dev
   ```

2. **Wait for the server** to be ready at `http://localhost:2024`

3. **Run this notebook** - it will auto-discover your assistant configuration

### Why This Architecture?

When you run `langgraph dev`, LangGraph connects to LangSmith's cloud infrastructure for storage.
This means:
- Memories persist across sessions and are shared between this notebook and LangSmith Studio
- No local PostgreSQL database required
- The assistant_id is auto-discovered, so this notebook works for anyone

## Setup

In [ ]:
import json
from datetime import datetime
from langgraph_sdk import get_sync_client

# Connect to local LangGraph server (must have `langgraph dev` running)
LANGGRAPH_URL = "http://localhost:2024"

try:
    client = get_sync_client(url=LANGGRAPH_URL)
    # Quick health check
    client.assistants.search(limit=1)
    print(f"Connected to LangGraph server at {LANGGRAPH_URL}")
except Exception as e:
    print(f"Could not connect to LangGraph server at {LANGGRAPH_URL}")
    print(f"Error: {e}")
    print("\nMake sure to run 'langgraph dev' from the deep-agent/ directory first.")
    raise SystemExit(1)

In [ ]:
# Auto-discover the assistant namespace
# This ensures the notebook works for anyone, regardless of their assistant_id

namespaces = client.store.list_namespaces()
filesystem_namespaces = [ns for ns in namespaces["namespaces"] if len(ns) == 2 and ns[1] == "filesystem"]

if filesystem_namespaces:
    NAMESPACE = filesystem_namespaces[0]
    ASSISTANT_ID = NAMESPACE[0]
    print(f"Auto-discovered assistant: {ASSISTANT_ID}")
    print(f"Using namespace: {NAMESPACE}")
else:
    print("No assistant namespace found.")
    print("\nThis usually means:")
    print("  1. You haven't run the agent yet in LangSmith Studio, OR")
    print("  2. The agent hasn't written any memories yet")
    print("\nTry running a query in LangSmith Studio first, then re-run this cell.")
    NAMESPACE = None
    ASSISTANT_ID = None

## Memory Namespace Structure

The agent uses a namespace of `[assistant_id, "filesystem"]` for persistent storage.

Within this namespace, memory files are stored as:
- `/website_quality.txt` - Website reliability ratings
- `/research_lessons.txt` - What approaches worked well
- `/source_notes.txt` - Notes about specific sources
- `/coding.txt` - Code mistakes and lessons

**Note**: The `/memories/` prefix from the agent's perspective is stripped by the CompositeBackend routing, so files are stored without it in the actual store.

## 1. View All Memories

In [ ]:
def list_all_memories():
    """List all memories in the agent's namespace."""
    if not NAMESPACE:
        print("No namespace configured. Run the setup cells first.")
        return []
    
    response = client.store.search_items(NAMESPACE, limit=100)
    items = response["items"]
    
    if not items:
        print("No memories found")
        return []
    
    print(f"Found {len(items)} memories:\n")
    for i, item in enumerate(items, 1):
        print(f"--- Memory {i} ---")
        print(f"Key: {item['key']}")
        print(f"Created: {item['created_at']}")
        print(f"Updated: {item['updated_at']}")
        
        # Handle content format (list of lines)
        value = item["value"]
        if isinstance(value.get("content"), list):
            content = "\n".join(value["content"])
        else:
            content = json.dumps(value, indent=2)
        
        if len(content) > 500:
            print(f"Content: {content[:500]}...")
        else:
            print(f"Content: {content}")
        print()
    
    return items

# List all memories
all_memories = list_all_memories()

## 2. View Specific Memory File

In [ ]:
def get_memory(key: str):
    """Get a specific memory by key."""
    if not NAMESPACE:
        print("No namespace configured. Run the setup cells first.")
        return None
    
    # Ensure key starts with /
    if not key.startswith("/"):
        key = f"/{key}"
    
    try:
        item = client.store.get_item(NAMESPACE, key)
        if item:
            print(f"Memory: {key}")
            print(f"Created: {item['created_at']}")
            print(f"Updated: {item['updated_at']}")
            
            value = item["value"]
            if isinstance(value.get("content"), list):
                content = "\n".join(value["content"])
            else:
                content = json.dumps(value, indent=2)
            
            print(f"\nContent:\n{content}")
            return item
        else:
            print(f"Memory '{key}' not found")
            return None
    except Exception as e:
        print(f"Error: {e}")
        return None

# Example: Get website quality memory
get_memory("website_quality.txt")

## 3. Save/Overwrite Memories

Use this to pre-seed the agent with memories or update existing ones.

In [ ]:
def save_memory(key: str, content: str):
    """Save or update a memory."""
    if not NAMESPACE:
        print("No namespace configured. Run the setup cells first.")
        return
    
    # Ensure key starts with /
    if not key.startswith("/"):
        key = f"/{key}"
    
    timestamp = datetime.now().isoformat()
    client.store.put_item(
        NAMESPACE,
        key,
        value={
            "content": content.split("\n"),
            "created_at": timestamp,
            "modified_at": timestamp,
        }
    )
    print(f"Saved memory: {key}")

In [ ]:
# Seed all 4 memory files with realistic content

# Memory 1: Website quality discoveries
save_memory("website_quality.txt", """# Website Quality Ratings
Last updated: 2025-03-05

## Discovered High Quality Sources
- reuters.com (5/5) - Fast, timestamped equity headlines; strong for M&A and earnings timing
- ft.com (4/5) - Good macro/regulatory context; cross-check figures against filings due to paywall summaries
- sec.gov/edgar (5/5) - Primary filings and 8-Ks; reliable event timestamps
- company investor relations sites (4/5) - Earnings releases, decks, transcripts; marketing tone but primary data
- bls.gov / bea.gov (5/5) - Official macro prints; cite release time in ET

## Sources That Disappointed
- seekingalpha.com (2/5) - Opinion-heavy; numbers often rounded or missing timestamps
- yahoo.com/finance aggregated news (3/5) - Rewrites with delayed timestamps; verify against originals
- marketbeat.com (2/5) - Promotional bias around analyst ratings
- twitter.com/x.com (2/5) - Sentiment only; not dependable for factual claims

## Domain-Specific Findings
- For earnings: start with 8-K + press release; use Reuters for corroborating headline times
- For guidance changes: check IR decks and follow-up 8-K exhibits; call transcripts add nuance
- For macro prints: use BLS/BEA/Fed releases; supplement with Reuters for market color
- For regulatory actions: use sec.gov, ftc.gov, and ec.europa.eu press rooms for primary statements
""")

# Memory 2: Research strategies that worked
save_memory("research_lessons.txt", """# Research Lessons Learned
Last updated: 2025-03-05

## Effective Search Patterns
- Use "TICKER earnings 8-K" and "TICKER investor relations presentation" to get primary documents fast
- Add `site:reuters.com TICKER` with the last two dates to capture timestamped headlines in the 48h window
- Combine sector term + "regulation" + current year to surface policy moves (e.g., "semiconductor export rules 2025")
- Pair company + supplier/peer names to uncover supply chain impacts
- Pull econ releases via "BLS CPI release" or "FOMC statement PDF" to land on official links

## What Didn't Work
- Generic "company news" queries return SEO blogs with stale dates
- Analyst note summaries without tickers miss context; need original note or wire copy
- Social clips about rumors rarely cite sources; avoid unless corroborated

## Verification Strategies
- Cross-check headline timestamp vs filing time to align with price move window
- For price reactions, anchor to local market hours and convert all times to GMT/ET consistently
- Require at least two independent wires for big claims (M&A, investigations, outages)
- Save raw links plus a short note on what was confirmed to speed credibility checks later
""")

# Memory 3: Specific source notes
save_memory("source_notes.txt", """# Source Notes

## reuters.com
- Datelines reliable; includes author + timestamp; good for quick double-source on filings
- Often provides context on sector peers and analyst quotes

## sec.gov/edgar
- Use 8-K for unscheduled news; 10-Q/10-K for numbers; timestamps are ET
- Download exhibits for slides; some data only lives in attachments

## company investor relations
- Press releases and slide decks usually post within minutes of EDGAR filing
- Earnings call transcripts (company-hosted or third-party) need verification against audio for quotes

## ft.com
- Strong for regulatory and geopolitical context affecting sectors
- Paywalled; summarize but verify figures against primary releases
""")

# Memory 4: Coding mistakes and lessons
save_memory("coding.txt", """# Coding Lessons
Last updated: 2025-03-05

## Common Pitfalls
- Normalize timestamps to timezone-aware UTC, then convert to ET/GMT for market context
- Align price series to market sessions before calculating event-window returns
- Trim outliers and low-liquidity periods; check volume to avoid false spikes
- Close matplotlib figures after saving to prevent memory leaks in repeated runs

## Debugging Tips
- Log dataframe shapes after merges; mismatched tickers often drop rows silently
- Plot price and volume around events to confirm the move is real
- When reading CSVs from scratchpad, use parse_dates to avoid string time issues
""")

print("\nAll seed memories created!")
list_all_memories()

## 4. Append to Existing Memory

Add new learnings to an existing memory file without overwriting.

In [ ]:
def append_to_memory(key: str, new_content: str):
    """Append content to an existing memory."""
    if not NAMESPACE:
        print("No namespace configured. Run the setup cells first.")
        return
    
    if not key.startswith("/"):
        key = f"/{key}"
    
    try:
        existing = client.store.get_item(NAMESPACE, key)
        if existing and "content" in existing["value"]:
            current_lines = existing["value"]["content"]
            new_lines = new_content.split("\n")
            updated_lines = current_lines + [""] + new_lines
            updated_content = "\n".join(updated_lines)
        else:
            updated_content = new_content
        
        save_memory(key, updated_content)
    except Exception:
        # File doesn't exist, create it
        save_memory(key, new_content)

In [ ]:
# Example: Append a new discovery to research lessons
append_to_memory("research_lessons.txt", """
## New Discovery (added manually)
- "Ticker + premarket" queries surface why moves start before the open; pair with time-filtered Reuters hits
- EDGAR 'Latest Filings' page is the fastest way to confirm unscheduled 8-Ks after-hours
""")

print("Appended to research_lessons.txt")

In [ ]:
get_memory("research_lessons.txt")

## 5. Delete Memories

In [ ]:
def delete_memory(key: str):
    """Delete a specific memory."""
    if not NAMESPACE:
        print("No namespace configured. Run the setup cells first.")
        return
    
    if not key.startswith("/"):
        key = f"/{key}"
    
    client.store.delete_item(NAMESPACE, key)
    print(f"Deleted: {key}")

def clear_all_memories():
    """Delete ALL memories (use with caution!)."""
    if not NAMESPACE:
        print("No namespace configured. Run the setup cells first.")
        return
    
    response = client.store.search_items(NAMESPACE, limit=100)
    items = response["items"]
    
    for item in items:
        client.store.delete_item(NAMESPACE, item["key"])
    
    print(f"Cleared {len(items)} memories")

# Example:
# delete_memory("coding.txt")  # Delete specific file
# clear_all_memories()  # Delete everything!

## 6. Quick Reference

```python
# List all memories
list_all_memories()

# Get specific memory
get_memory("website_quality.txt")

# Save/overwrite memory
save_memory("coding.txt", "# My content\n- bullet 1\n- bullet 2")

# Append to memory
append_to_memory("research_lessons.txt", "## New section\n- new lesson")

# Delete memory
delete_memory("coding.txt")

# Clear all (careful!)
clear_all_memories()
```